In [390]:
%reset -f

In [391]:
from IPython.display import display, HTML
from sklearn import metrics
from sklearn import cross_validation
from scipy import interp
import pandas as pd
import numpy as np
import os

import sys
sys.path.insert(1,"../../src/")
from TypeFeatImputer import TypeFeatImputer
from i_score_parallel import i_score
from TypeFeatFS import DiscreteFS, ContinuousFS
from TypeFeatFilter import DiscreteFilter, ContinuousFilter
from OutlierFiltering import OutlierFiltering
from typeFeat_score import typeFeat_score

from pylab import *
import seaborn as sns
import matplotlib.pyplot as plt


np.set_printoptions(suppress=True)
pd.options.display.float_format = '{:,.4f}'.format
plt.style.use('classic')

%matplotlib inline

In [581]:
ddf = []
pathDir = os.path.join('resources','results') 
path, dirs, files = os.walk(pathDir).next()
file_count = len(files)

for f in files:
    if "summary" not in str(f):
        ddf.append(pd.read_pickle(os.path.join('resources','results', str(f) )))
    
dfAll = pd.concat(ddf)
print dfAll.shape

(328, 35)


In [582]:
dfAll.cv_f1_mean = pd.to_numeric(dfAll.cv_f1_mean)
dfAll.cv_auc_mean = pd.to_numeric(dfAll.cv_auc_mean)
dfAll.cv_sens_mean = pd.to_numeric(dfAll.cv_sens_mean)
dfAll.cv_spec_mean = pd.to_numeric(dfAll.cv_spec_mean)

dfAll.cv_f1_std = pd.to_numeric(dfAll.cv_f1_std)
dfAll.cv_auc_std = pd.to_numeric(dfAll.cv_auc_std)
dfAll.cv_sens_std = pd.to_numeric(dfAll.cv_sens_std)
dfAll.cv_spec_std = pd.to_numeric(dfAll.cv_spec_std)
dfAll.exp = pd.to_numeric(dfAll.exp)

dfAll["cv_auc"] = dfAll["cv_auc_mean"].round(3).astype(str).str.cat(dfAll["cv_auc_std"].round(3).astype(str), sep="+/-")
dfAll["cv_f1"] = dfAll["cv_f1_mean"].round(3).astype(str).str.cat(dfAll["cv_f1_std"].round(3).astype(str), sep="+/-")
dfAll["cv_sens"] = dfAll["cv_sens_mean"].round(3).astype(str).str.cat(dfAll["cv_sens_std"].round(3).astype(str), sep="+/-")
dfAll["cv_spec"] = dfAll["cv_spec_mean"].round(3).astype(str).str.cat(dfAll["cv_spec_std"].round(3).astype(str), sep="+/-")
dfAll["disease"] = dfAll["name"].str.slice(0,dfAll["name"].str.find("_").values[0])

print
print dfAll.columns.tolist()
print dfAll.name.unique()
print dfAll["disease"].unique()
print dfAll.shape


['exp', 'name', 'size_tr', 'fs', 'sm', 'cls', 'metric', 'params', 'tr_sens', 'tr_spec', 'tr_auc', 'tr_prec', 'tr_rec', 'tr_f1', 'cv_sens_mean', 'cv_sens_std', 'cv_spec_mean', 'cv_spec_std', 'cv_auc_mean', 'cv_auc_std', 'cv_prec_mean', 'cv_prec_std', 'cv_rec_mean', 'cv_rec_std', 'cv_f1_mean', 'cv_f1_std', 'test_sens', 'test_spec', 'test_auc', 'test_rec', 'test_prec', 'test_f1', 'cm_test', 'time', 'pipeline', 'cv_auc', 'cv_f1', 'cv_sens', 'cv_spec', 'disease']
['all_extended_last_early_readmission_vs_none'
 'all_reduced_last_early_readmission_vs_none'
 'all_reduced_last_all_readmisssion_vs_none'
 'all_extended_last_all_readmisssion_vs_none'
 'all_extended_extra_last_early_readmission_vs_none'
 'Diabetis_extended_extra_diag_1_last_all_readmisssion_vs_none'
 'all_extended_extra_last_all_readmisssion_vs_none'
 'Circulatory_extended_extra_diag_1_last_all_readmisssion_vs_none'
 'Digestive_extended_extra_diag_1_last_all_readmisssion_vs_none'
 'Genitourinary_extended_extra_diag_1_last_all_read

In [585]:
typeEncounter = "last" # ['first','last']
typeHypothesis = "early_readmission_vs_none" # ['all_readmisssion_vs_none','early_readmission_vs_none']
typeDataFeatures = "extended_extra_diag_1" # ["reduced","extended','extended_extra','extended_extra_diag_1','extended_extra_diag_3']
    #Extended_Extra_diag_1 -> Extended extra columns with disease in diag_3
    #Extended_Extra_diag_1 -> Extended extra columns with disease in diag_1
    #Extended_Extra -> Extended extra of columns
    #Extended -> Extended columns
    #Reduced -> minimum set of columns 
typeDataExperiment = "all" #["all", "disease"] 
    #all -> No filter disease column
    #disease -> Filter/Remove disease column and keep only rows with diagnosis column == 1  

name = typeDataExperiment + "_" + typeDataFeatures + "_" +  typeEncounter + "_" + typeHypothesis

print name

all_extended_extra_diag_1_last_early_readmission_vs_none


In [586]:
#Sort and list all pipelines
dfAll.sort_values(["cv_auc_mean"], ascending=False,inplace=True)
print dfAll[dfAll['name'].str.contains(name)].shape

dfAll[dfAll['name'].str.contains(name)][["disease","fs","metric","cls","params",
    "tr_auc","tr_sens","tr_spec",
    "cv_auc_mean","cv_f1_mean","cv_sens_mean","cv_spec_mean",
    "test_auc","test_f1","test_sens","test_spec"]]

(0, 40)


,disease,fs,metric,cls,params,tr_auc,tr_sens,tr_spec,cv_auc_mean,cv_f1_mean,cv_sens_mean,cv_spec_mean,test_auc,test_f1,test_sens,test_spec


In [579]:
#Filter pipelines less performance than random
dfAllAux = dfAll[np.logical_and(np.logical_and(dfAll['name'].str.contains(name), dfAll["cv_sens_mean"] > 0.5),
                     dfAll['cv_spec_mean']> 0.5)].copy()
dfAllAux[[
    "disease","sm","fs","metric","cls","params",
    "tr_auc","tr_sens","tr_spec",
    "cv_auc_mean","cv_f1_mean","cv_sens_mean","cv_spec_mean",
    "test_auc","test_f1","test_sens","test_spec"]]

,disease,sm,fs,metric,cls,params,tr_auc,tr_sens,tr_spec,cv_auc_mean,cv_f1_mean,cv_sens_mean,cv_spec_mean,test_auc,test_f1,test_sens,test_spec
0,Dia,none,none,recall,logReg,"[balanced, 1e-05, l2]",0.6439,0.6751,0.6126,0.7087,0.6622,0.6925,0.5856,0.6727,0.6592,0.7763,0.5690
0,Dia,none,none,f1,rf,"[gini, 4, 250]",0.6901,0.5932,0.7869,0.7048,0.7536,0.5151,0.7495,0.6832,0.7697,0.6118,0.7545
0,Dia,none,none,recall,rf,"[gini, 4, 300]",0.6880,0.5932,0.7828,0.7042,0.7541,0.5132,0.7507,0.6843,0.7678,0.6184,0.7503
0,Dia,none,rfe_rf_fs,f1,logReg,"[20, balanced, 10, 0.1, l2]",0.6553,0.5819,0.7287,0.7039,0.7288,0.5792,0.6995,0.6917,0.7344,0.6974,0.6860
0,Dia,none,combine_fs,f1,rf,"[entropy, 4, 250, 50]",0.6813,0.6045,0.7580,0.7018,0.7388,0.5472,0.7206,0.6937,0.7555,0.6645,0.7229
0,Dia,none,combine_fs,f1,logReg,"[balanced, 0.01, l2, 40]",0.6520,0.5960,0.7079,0.7016,0.7222,0.6000,0.6857,0.7065,0.7363,0.7303,0.6828
0,Dia,none,none,f1,logReg,"[balanced, 0.1, l2]",0.6872,0.6384,0.7359,0.7010,0.7303,0.5774,0.7020,0.6790,0.7319,0.6711,0.6870
0,Dia,none,lasso_fs,recall,rf,"[gini, 0.1, 4, 300]",0.6988,0.6243,0.7734,0.7010,0.7423,0.5472,0.7257,0.6883,0.7660,0.6316,0.7450
0,Dia,none,lasso_fs,f1,rf,"[gini, 0.1, 4, 300]",0.6988,0.6243,0.7734,0.7010,0.7423,0.5472,0.7257,0.6883,0.7660,0.6316,0.7450
0,Dia,none,combine_fs,recall,logReg,"[balanced, 1e-05, l2, 40]",0.6454,0.6610,0.6298,0.6973,0.6812,0.6717,0.6147,0.7099,0.6858,0.8224,0.5975


In [549]:
dfAllAux[[
    "metric","fs","cls","params",
    "tr_auc","tr_sens","tr_spec",
    "cv_auc_mean","cv_f1_mean","cv_sens_mean","cv_spec_mean",
    "test_auc","test_f1","test_sens","test_spec"]].to_latex()

u'\\begin{tabular}{llllllllrrrrllll}\n\\toprule\n{} &       metric &    fs &     cls &                  params & tr\\_auc & tr\\_sens & tr\\_spec &  cv\\_auc\\_mean &  cv\\_f1\\_mean &  cv\\_sens\\_mean &  cv\\_spec\\_mean & test\\_auc & test\\_f1 & test\\_sens & test\\_spec \\\\\n\\midrule\n0 &  f1\\_weighted &  none &      rf &         [gini, 12, 400] & 0.6936 &  0.6732 &  0.7140 &       0.6536 &      0.6153 &        0.5659 &        0.6462 &   0.6101 &  0.6175 &    0.5833 &    0.6369 \\\\\n0 &           f1 &  none &      rf &          [gini, 8, 350] & 0.6234 &  0.6062 &  0.6407 &       0.6493 &      0.6121 &        0.5836 &        0.6271 &   0.6077 &  0.6131 &    0.5947 &    0.6207 \\\\\n0 &       recall &  none &      rf &          [gini, 8, 350] & 0.6236 &  0.6064 &  0.6409 &       0.6493 &      0.6121 &        0.5836 &        0.6271 &   0.6078 &  0.6131 &    0.5947 &    0.6208 \\\\\n0 &  f1\\_weighted &  none &  logReg &   [balanced, 0.005, l2] & 0.5978 &  0.5673 &  0.6282 &      

#### Grouped by Diseases

In [580]:
dfAllAux = dfAllAux.sort_values(["disease","cv_auc_mean"], ascending=False,inplace=False)
dfAllAux = dfAllAux[np.logical_and(dfAllAux["cv_sens_mean"] > 0.5,dfAllAux['cv_spec_mean']> 0.5)]
dfAllAux = dfAllAux.groupby(["disease"]).first()
dfAllAux[["sm","fs","metric","cls","params",
    "tr_auc","tr_sens","tr_spec",
    "cv_auc_mean","cv_f1_mean","cv_sens_mean","cv_spec_mean",
    "test_auc","test_f1","test_sens","test_spec"]]

,sm,fs,metric,cls,params,tr_auc,tr_sens,tr_spec,cv_auc_mean,cv_f1_mean,cv_sens_mean,cv_spec_mean,test_auc,test_f1,test_sens,test_spec
disease,,,,,,,,,,,,,,,,
Cir,none,none,f1,logReg,"[balanced, 0.05, l1]",0.6150,0.5833,0.6467,0.6411,0.6802,0.5686,0.6352,0.5902,0.6745,0.5498,0.6306
Dia,none,none,recall,logReg,"[balanced, 1e-05, l2]",0.6439,0.6751,0.6126,0.7087,0.6622,0.6925,0.5856,0.6727,0.6592,0.7763,0.5690
Dig,none,combine_fs,recall,logReg,"[balanced, 0.0005, l2, 50]",0.6318,0.6228,0.6409,0.6469,0.7003,0.5553,0.6470,0.5918,0.6923,0.5479,0.6357
Gen,none,none,recall,logReg,"[balanced, 0.05, l1]",0.6813,0.7225,0.6400,0.6533,0.6860,0.6222,0.6236,0.6293,0.6916,0.6292,0.6294
Mus,none,rfe_rf_fs,f1,logReg,"[15, balanced, 15, 0.1, l2]",0.6699,0.7778,0.5620,0.6804,0.6605,0.7032,0.5720,0.6048,0.6499,0.6458,0.5638
Neo,none,none,recall,logReg,"[balanced, 0.0001, l2]",0.6209,0.6824,0.5593,0.6263,0.6547,0.6318,0.5682,0.5879,0.6403,0.6250,0.5509
Poi,none,none,f1,logReg,"[balanced, 0.05, l1]",0.6343,0.6893,0.5793,0.6562,0.6351,0.6804,0.5569,0.6372,0.6403,0.7172,0.5572
Res,none,none,f1,logReg,"[balanced, 0.1, l2]",0.6632,0.6409,0.6855,0.6486,0.7045,0.5431,0.6656,0.5689,0.6863,0.4903,0.6476


In [531]:
print dfAllAux[["metric","cls","params","tr_auc","tr_sens","tr_spec",
    "cv_auc_mean","cv_f1_mean","cv_sens_mean","cv_spec_mean","test_auc",
                "test_f1","test_sens","test_spec"]].to_latex()

\begin{tabular}{llllrrrrrrrrrrr}
\toprule
{} &  metric &     cls &                  params &  tr\_auc &  tr\_sens &  tr\_spec &  cv\_auc\_mean &  cv\_f1\_mean &  cv\_sens\_mean &  cv\_spec\_mean &  test\_auc &  test\_f1 &  test\_sens &  test\_spec \\
disease &         &         &                         &         &          &          &              &             &               &               &           &          &            &            \\
\midrule
Cir     &      f1 &  logReg &    [balanced, 0.05, l1] &  0.6150 &   0.5833 &   0.6467 &       0.6411 &      0.6802 &        0.5686 &        0.6352 &    0.5902 &   0.6745 &     0.5498 &     0.6306 \\
Dia     &  recall &  logReg &   [balanced, 1e-05, l2] &  0.6439 &   0.6751 &   0.6126 &       0.7087 &      0.6622 &        0.6925 &        0.5856 &    0.6727 &   0.6592 &     0.7763 &     0.5690 \\
Dig     &  recall &  logReg &  [balanced, 0.0005, l2] &  0.6458 &   0.6667 &   0.6250 &       0.6469 &      0.6911 &        0.5903 &        0.6

#### Save results

In [550]:
import time

fname = "summary" + "_" + name + "_" + time.strftime("%Y%m%d-%H%M%S") + ".pkl"
dfAllAux.to_pickle(os.path.join(pathDir, fname))
print fname

summary_reduced_last_all_readmisssion_vs_none_20171025-115456.pkl


In [551]:
dd = pd.read_pickle(os.path.join(pathDir,fname))
print dd.columns

Index([u'exp', u'name', u'size_tr', u'fs', u'sm', u'cls', u'metric', u'params',
       u'tr_sens', u'tr_spec', u'tr_auc', u'tr_prec', u'tr_rec', u'tr_f1',
       u'cv_sens_mean', u'cv_sens_std', u'cv_spec_mean', u'cv_spec_std',
       u'cv_auc_mean', u'cv_auc_std', u'cv_prec_mean', u'cv_prec_std',
       u'cv_rec_mean', u'cv_rec_std', u'cv_f1_mean', u'cv_f1_std',
       u'test_sens', u'test_spec', u'test_auc', u'test_rec', u'test_prec',
       u'test_f1', u'cm_test', u'time', u'pipeline', u'cv_auc', u'cv_f1',
       u'cv_sens', u'cv_spec', u'disease'],
      dtype='object')
